In [1]:
import json
import os
from argparse import ArgumentParser

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm

In [59]:
DATA_ROOT='/hdd/sdfstudio/data/colmap_sdfstudio/room_raw'
path_transform = os.path.join(DATA_ROOT, 'transforms.json')
dir_normal = os.path.join(DATA_ROOT, 'normal')
dir_mask = os.path.join(DATA_ROOT, 'mask')


In [60]:
def align_filter_normal(path_normals, path_masks, c2ws):
    print('Aligning and filering normal ...')
    normals = []
    for i in tqdm(range(len(path_normals))):
        path_normal = path_normals[i]
        path_mask = path_masks[i]
        c2w = c2ws[i] # OpneGL
        mask = np.array(Image.open(path_mask)) > 0 # (h, w)
        if not mask.any():
            continue
        normal = np.array(Image.open(path_normal))/255.0 # (h, w, 3)
        normal = normal * 2 - 1 
        normal[..., 1:3] *= -1 
        R_c2w = c2w[:3, :3]
        normal_world = normal @ R_c2w.T
        normal_world = normal_world / np.linalg.norm(normal_world, axis=-1, keepdims=True)
        normal_world = normal_world[mask]
        normals.append(normal_world)
    normals = np.concatenate(normals, axis=0)
    return normals

In [61]:
with open(path_transform, 'r') as f:
    transform = json.load(f)

frames = transform['frames']
c2ws = np.array([frames[i]['transform_matrix'] for i in range(len(frames))])
paths = np.array([frames[i]['file_path'] for i in range(len(frames))])
argsort = np.argsort(paths)
c2ws = c2ws[argsort]

paths_normal = sorted([os.path.join(dir_normal, f) for f in os.listdir(dir_normal)])
paths_mask = sorted([os.path.join(dir_mask, f) for f in os.listdir(dir_mask)])
assert len(paths_normal) == len(paths_mask) == len(c2ws)

normals = align_filter_normal(paths_normal, paths_mask, c2ws)

Aligning and filering normal ...


  0%|          | 0/311 [00:00<?, ?it/s]

100%|██████████| 311/311 [00:11<00:00, 27.84it/s]


In [62]:
# RANSAC
n_iter = 100
n_sample = 10000
threshold = 0.99
best_inliers = 0
best_normal = None
for i in tqdm(range(n_iter)):
    idx = np.random.choice(len(normals), n_sample, replace=False)
    normal = np.mean(normals[idx], axis=0)
    normal /= np.linalg.norm(normal)
    inliers = normals @ normal > threshold
    n_inliers = np.sum(inliers)
    if n_inliers > best_inliers:
        print(f'Inliers: {n_inliers/len(normals)}')
        best_inliers = n_inliers
        best_normal = normal

  1%|          | 1/100 [00:00<00:49,  2.01it/s]

Inliers: 0.5626546746606013


  3%|▎         | 3/100 [00:01<00:52,  1.84it/s]

Inliers: 0.5760104827522241


  5%|▌         | 5/100 [00:02<00:53,  1.77it/s]

Inliers: 0.5967593474401561


 10%|█         | 10/100 [00:05<00:49,  1.82it/s]

Inliers: 0.6110161176101199


100%|██████████| 100/100 [00:54<00:00,  1.85it/s]


In [63]:
print('Best normal:', best_normal)
print('Best inliers rate:', best_inliers / len(normals))

Best normal: [-0.90442986 -0.08648835  0.41776356]
Best inliers rate: 0.6110161176101199


In [64]:
# Get the rotation matrix that aligns the best normal to (0, 0, 1)
import cv2
v = np.array([0, 0, 1])

normal_temp = best_normal.copy()
R_all = np.eye(3)
for i in range(100):
    axis = np.cross(normal_temp, v)
    angle = np.arccos(np.dot(normal_temp, v))
    R = cv2.Rodrigues(axis * angle)[0]
    normal_temp = R @ normal_temp
    R_all = R @ R_all

normal_aligned = R_all @ best_normal
print('normal_temp:\n', normal_temp)
print('Rotation:\n', R_all)
print('Aligned normal:\n', normal_aligned)
# print(R_all.T @ R_all)
save_path = os.path.join(DATA_ROOT, 'rotation_aligned.npy')
np.save(save_path, R_all)

normal_temp:
 [-8.96167510e-03 -8.56982413e-04  9.99959476e-01]
Rotation:
 [[ 0.43112807 -0.05439979  0.90064935]
 [-0.05439979  0.99479789  0.08612683]
 [-0.90064935 -0.08612683  0.42592595]]
Aligned normal:
 [-8.96167510e-03 -8.56982413e-04  9.99959476e-01]
